In [1]:
from transformers import GPTNeoXForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model
import torch
import torch.nn as nn

/home/vmasti/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# first get the finetuned small model

small_name = "EleutherAI/pythia-70m"
large_name = "EleutherAI/pythia-410m"
small_adapter = "weight/pythia_70m_lora_r=64"

model_small_pt = GPTNeoXForCausalLM.from_pretrained(small_name)
model_small_ft = PeftModel.from_pretrained(model_small_pt, small_adapter)

In [3]:
model_large_pt = GPTNeoXForCausalLM.from_pretrained(large_name)
model_large_pt

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [4]:
model_small_ft

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50304, 512)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-5): 6 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Li

In [37]:
# config_mora = LoraConfig(
#     use_mora=True, 
#     mora_type=6,  # RoPE for small rank
#     r=64, 
#     target_modules=["query_key_value"], 
#     lora_dropout=0.05, 
#     task_type="CAUSAL_LM"
#     # MoRA does not use lora_alpha
# )
# model = GPTNeoXForCausalLM.from_pretrained(
#     "EleutherAI/pythia-410m"
# )
# model = get_peft_model(model, config_mora)

In [10]:
# expand module

def expand_lora_copy(old_module, new_in, new_out):
    new_module = nn.Linear(new_in, new_out, bias=old_module.bias is not None)
    nn.init.zeros_(new_module.weight)
    
    with torch.no_grad():
        new_module.weight[:old_module.out_features, :old_module.in_features].copy_(old_module.weight)
        
        for i in range(old_module.in_features, new_in):
            new_module.weight[:old_module.out_features, i].copy_(old_module.weight[:, i % old_module.in_features])
        
        for j in range(old_module.out_features, new_out):
            new_module.weight[j, :old_module.in_features].copy_(old_module.weight[j % old_module.out_features, :])
    
    if old_module.bias is not None:
        new_module.bias[:old_module.out_features].copy_(old_module.bias)
    
    return new_module


def expand_lora_padding(old_module, new_in, new_out):
    new_module = nn.Linear(new_in, new_out, bias=old_module.bias is not None)
    nn.init.zeros_(new_module.weight)

    with torch.no_grad():
        new_module.weight[:old_module.out_features, :old_module.in_features].copy_(old_module.weight)

    if old_module.bias is not None:
        new_module.bias[:old_module.out_features].copy_(old_module.bias)
    
    return new_module

In [11]:
new_in_a, new_out_a = 1024, 64
new_in_b, new_out_b = 64, 3072

for name, module in model_small_ft.named_modules():
    if name.endswith("lora_A"):
        new_module = expand_lora_padding(module.default, new_in_a, new_out_a)
        parts = name.split('.')
        parent_module = model_small_ft
        for part in parts[:-1]:
            parent_module = getattr(parent_module, part)
        
        setattr(parent_module, parts[-1], nn.ModuleDict({"default": new_module}))

    elif name.endswith("lora_B"):
        new_module = expand_lora_padding(module.default, new_in_b, new_out_b)
        parts = name.split('.')
        parent_module = model_small_ft
        for part in parts[:-1]:
            parent_module = getattr(parent_module, part)
        
        setattr(parent_module, parts[-1], nn.ModuleDict({"default": new_module}))

In [12]:
# save the expanded model

model_small_ft.save_pretrained("weight/pythia_70m_lora_expanded_padding_r=64")

In [13]:
model_small_ft

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50304, 512)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-5): 6 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Li